In [59]:
import json
import numpy as np
import tensorflow as tf
from keras.layers import Conv2D,Dropout,MaxPool2D,BatchNormalization,Dense,Flatten
from keras.models import Sequential
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [37]:
f = open('planes.json')
planesnet = json.load(f)
f.close()

In [11]:
print(np.unique(planesnet.labels))

[0 1]


In [47]:
#Preprocessing
X = np.array(planesnet['data'])/255.
X = X.reshape(-1,20,20,3)
Y = np.array(planesnet['labels'])
Y = to_categorical(Y,2)

In [60]:
#Data Preprocessing

X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=0.8,random_state=1)

In [56]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',activation='relu',kernel_initializer='he_uniform',input_shape=[20,20,3]))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))

model.add(Flatten())

model.add(Dense(1024,activation='relu'))
model.add(Dropout((0.5)))
model.add(Dense(512,activation='relu'))
model.add(Dropout((0.5)))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [57]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 20, 20, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 20, 20, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 10, 10, 64)        256       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)             

In [62]:
model.fit(X_train,y_train,epochs=10,shuffle=True,batch_size=128,verbose=1,validation_data=(X_test,y_test))

Train on 25600 samples, validate on 6400 samples
Epoch 1/10
25600/25600 [==============================] - 88s 3ms/step - loss: 0.1944 - accuracy: 0.9254 - val_loss: 0.2636 - val_accuracy: 0.8611
Epoch 2/10
25600/25600 [==============================] - 94s 4ms/step - loss: 0.1037 - accuracy: 0.9618 - val_loss: 0.1364 - val_accuracy: 0.9455
Epoch 3/10
25600/25600 [==============================] - 86s 3ms/step - loss: 0.0789 - accuracy: 0.9719 - val_loss: 0.0987 - val_accuracy: 0.9641
Epoch 4/10
25600/25600 [==============================] - 80s 3ms/step - loss: 0.0629 - accuracy: 0.9780 - val_loss: 0.0895 - val_accuracy: 0.9673
Epoch 5/10
25600/25600 [==============================] - 80s 3ms/step - loss: 0.0519 - accuracy: 0.9810 - val_loss: 0.0817 - val_accuracy: 0.9725
Epoch 6/10
25600/25600 [==============================] - 73s 3ms/step - loss: 0.0444 - accuracy: 0.9834 - val_loss: 0.0722 - val_accuracy: 0.9750
Epoch 7/10
25600/25600 [==============================] - 73s 3ms/ste

In [65]:
from keras.models import model_from_json
model_json = model.to_json()
with open("./Model/PlaneModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Models/PlaneModel.h5")

In [79]:
json_file = open('./Models/PlaneModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Models/PlaneModel.h5")
print("Loaded model from disk")


Loaded model from disk


In [80]:
result = loaded_model.predict(X_test[500].reshape(1,20,20,3))
print("Original = {} Predicted = {}".format(y_test[500],np.round(result)))

Original = [0. 1.] Predicted = [[0. 1.]]


In [81]:
print(X_test.shape)
print(X_test[0].shape)

(6400, 20, 20, 3)
(20, 20, 3)
